In [ ]:
import mne
from pathlib import Path
import sys
import numpy as np
import torch
import time
project_root = Path.cwd().parent
sys.path.append(str(project_root))

import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)

from GEDAI import gedai, batch_gedai

In [ ]:
original_output_tim = torch.load("test.pt")

In [ ]:
leadfield = torch.load("./leadfield_calibrated/refCov_GSN_HydroCel_129.pt")
leadfield.shape

In [ ]:
sfreq = 100.0

In [ ]:
hbn_cleaned = batch_gedai(original_output_tim.unsqueeze(0), sfreq, leadfield=leadfield,
                         denoising_strength = "auto", epoch_size = 1.0, verbose_timing=True)
print(hbn_cleaned.shape)
plot_eeg(hbn_cleaned.squeeze(0), None, original_output_tim, title="EEG")

In [ ]:
def plot_eeg(
    signal,
    matlab_clean_signal,
    raw_signal,
    title: str,
    fs = 100,
    L = 10
):
    try:
        y = signal.detach().cpu().float().numpy()
    except:
        y = signal
    if matlab_clean_signal is not None:
        try:
            y_ = matlab_clean_signal.detach().cpu().float().numpy()
        except:
            y_ = matlab_clean_signal
    try:
        x = raw_signal.detach().cpu().float().numpy()
    except:
        x = raw_signal
    L = 5
    t = np.arange(y.shape[1]) / float(fs)

    fig, axes = plt.subplots(L, 1, figsize=(10, 1.8*L*3), sharex=True)

    for i in range(L):
        try:
            axes[i].plot(t, x[i], color='gray', label='Raw EEG', linewidth=1)
        except:
            pass
        axes[i].plot(t, y[i], color='blue', label='Cleaned By Python', linewidth=1)
        if matlab_clean_signal is not None:
            axes[i].plot(t, y_[i], color='red', label='Cleaned By Matlab', linewidth=1)
        axes[i].set_ylabel(f"Channel {i}")
        if i == 0:
            axes[i].legend(loc='upper right')

    axes[-1].set_xlabel("Time (s)")
    if title:
        fig.suptitle(title, y=0.995)
    fig.tight_layout()
    plt.show()

In [ ]:
hbn_cleaned = batch_gedai(original_output_tim.unsqueeze(0) * 1, 100, denoising_strength="auto", 
                          epoch_size=1.0, 
                          leadfield=leadfield, device="cpu" ,verbose_timing=True)

In [ ]:

sfreq_hbn = 100.0
original_output_tim = torch.load("test.pt")
leadfield = torch.load("./leadfield_calibrated/refCov_GSN_HydroCel_129.pt")
hbn_cleaned = batch_gedai(original_output_tim.unsqueeze(0), 100.0, "auto", 0.2, 
                          leadfield, device="cpu" ,verbose_timing=True)
hbn_cleaned = hbn_cleaned.squeeze(0)
plot_eeg(hbn_cleaned[:, :200], None, original_output_tim, title="EEG")